<a href="https://colab.research.google.com/github/simsekergun/RandomlyRotated/blob/main/Classification_RandomlyRotatedScatterer_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report
import time
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
url1 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM1.csv'
df1 = pd.read_csv(url1, index_col=0)

url2 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM2.csv'
df2 = pd.read_csv(url2, index_col=0)

url3 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM3.csv'
df3 = pd.read_csv(url3, index_col=0)

url4 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM4.csv'
df4 = pd.read_csv(url4, index_col=0)

url5 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM5.csv'
df5 = pd.read_csv(url5, index_col=0)

url6 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM6.csv'
df6 = pd.read_csv(url6, index_col=0)

url7 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM7.csv'
df7 = pd.read_csv(url7, index_col=0)

url8 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM8.csv'
df8 = pd.read_csv(url8, index_col=0)

url9 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM9.csv'
df9 = pd.read_csv(url9, index_col=0)

url10 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM10.csv'
df10 = pd.read_csv(url10, index_col=0)

url11 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM11.csv'
df11 = pd.read_csv(url11, index_col=0)

url12 = 'https://raw.githubusercontent.com/simsekergun/RandomlyRotated/refs/heads/main/MNIST_rot_SIM12.csv'
df12 = pd.read_csv(url12, index_col=0)

df_all = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12], axis=0)

# Reset the index
df_all = df_all.reset_index(drop=True)

print(f"Combined Shape: {df_all.shape}")

# View the first few rows
df_all.head()

Combined Shape: (12000, 65)


,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,8.599444e-20,-6.231016e-19,-6.933704e-19,-2.736925e-19,4.245287e-20,1.006229e-19,9.058320e-20,1.892554e-19,1.105886e-19,-2.186010e-19,...,-1.513291e-18,-1.553155e-18,-1.459761e-18,-1.688467e-19,1.982808e-18,6.879230e-19,-1.795921e-18,-4.888217e-19,1.745913e-18,5
1,3.137589e-20,-6.429994e-19,-8.023542e-19,-5.501632e-19,-4.509170e-19,-6.149811e-19,-6.402798e-19,-2.548319e-19,6.587843e-20,3.990638e-19,...,-2.246684e-18,-2.292242e-18,-1.992845e-18,-4.320242e-19,1.900514e-18,6.989732e-19,-1.842764e-18,-6.105473e-19,1.613595e-18,0
2,-1.326605e-20,-4.580125e-19,-2.876271e-19,8.287229e-20,1.161408e-19,-2.394476e-19,-4.567253e-19,-3.680656e-19,-1.405961e-19,1.693667e-19,...,-1.867904e-18,-2.064785e-18,-2.134977e-18,-7.622166e-19,1.709335e-18,8.240770e-19,-1.593208e-18,-5.260953e-19,1.548896e-18,4
3,1.155195e-19,-2.451452e-19,-2.715376e-19,-1.088131e-19,5.316636e-20,2.419866e-19,2.123935e-19,4.197650e-20,-1.450483e-19,-2.382157e-19,...,-1.531262e-18,-1.838182e-18,-1.980286e-18,-6.558416e-19,1.871363e-18,7.939170e-19,-1.757816e-18,-7.118855e-19,1.296885e-18,1
4,-5.492622e-20,-7.036797e-19,-4.647136e-19,-4.597923e-20,1.422098e-19,1.169888e-19,-1.251926e-20,-7.236105e-20,3.206697e-20,1.285730e-19,...,-1.899248e-18,-1.930469e-18,-1.751551e-18,-4.451874e-19,1.822326e-18,7.606922e-19,-1.651533e-18,-4.209629e-19,1.823976e-18,9


In [3]:
# 2. Separate Features (X) and Target (y)
# We take the first 64 columns for X
X = df_all.iloc[:, 0:64].values

# We take the last column for y (index -1)
# We convert it to integers
y = df_all.iloc[:, -1].astype(int).values

# 3. Create the 90% Training and 10% Testing split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, train_size=0.90, random_state=42
)

# Verify the shapes
print(f"Total samples: {len(df_all)}")
print(f"X_train shape: {X_train.shape}") # Should be (N*0.9, 64)
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape:  {X_test.shape}")  # Should be (N*0.1, 64)
print(f"y_test shape:  {y_test.shape}")

Total samples: 12000
X_train shape: (10800, 64)
y_train shape: (10800,)
X_test shape:  (1200, 64)
y_test shape:  (1200,)


In [4]:
# 1. Initialize the scaler
scaler = StandardScaler()

# 2. Fit on training data and transform both sets
# to ensure both sets have a mean of 0 and a standard deviation of 1
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Reshape for the CNN (16 positions, 4 components)
X_train_cnn = X_train_scaled.reshape(-1, 16, 4)
X_test_cnn = X_test_scaled.reshape(-1, 16, 4)

## SVM

Initial Test (before hyperparameter tuning)


In [5]:
# 1. Prepare Data: SVMs expect (n_samples, n_features)
# Flattening the (16, 4) input into a 64-dimensional vector
X_train_flat = X_train_cnn.reshape(X_train_cnn.shape[0], -1)
X_test_flat = X_test_cnn.reshape(X_test_cnn.shape[0], -1)

# 2. Scaling: Crucial for SVM performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

# 3. Define and Train the SVM
# Using RBF kernel to capture non-linear relationships
svm_model = svm.SVC(
    kernel='rbf',
    C=1.0,          # Regularization parameter
    gamma='scale',  # Kernel coefficient
    probability=True
)

# --- Start Training Timer ---
start_train = time.time()
svm_model.fit(X_train_scaled, y_train)
end_train = time.time()
# -----------------------------

# --- Start Inference Timer ---
start_test = time.time()
y_pred = svm_model.predict(X_test_scaled)
end_test = time.time()
# -----------------------------

# 4. Evaluation
accuracy = accuracy_score(y_test, y_pred)

print(f"Training Time: {end_train - start_train:.4f} seconds")
print(f"Inference Time: {end_test - start_test:.4f} seconds")
print(f"SVM Test Accuracy: {accuracy*100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Training Time: 24.2433 seconds
Inference Time: 1.0437 seconds
SVM Test Accuracy: 80.75%

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.92      0.92       110
           1       0.87      0.96      0.91       144
           2       0.76      0.78      0.77       115
           3       0.77      0.80      0.78       114
           4       0.76      0.72      0.74       125
           5       0.77      0.72      0.75       108
           6       0.79      0.90      0.84       110
           7       0.88      0.83      0.86       136
           8       0.73      0.65      0.69       108
           9       0.78      0.76      0.77       130

    accuracy                           0.81      1200
   macro avg       0.80      0.80      0.80      1200
weighted avg       0.81      0.81      0.81      1200



In [6]:
param_grid = {
    "C": [0.1, 1, 10, 100, 1000],
    "gamma": [1e-3, 1e-2, 1e-1, 1, "scale", "auto"]
}

grid = GridSearchCV(
    svm.SVC(kernel="rbf"),
    param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

start_grid = time.time()
grid.fit(X_train_scaled, y_train)
end_grid = time.time()

best_svm = grid.best_estimator_
print(f"Grid Search Time: {end_grid - start_grid:.4f} seconds")
print("Best parameters:", grid.best_params_)


Grid Search Time: 935.9293 seconds
Best parameters: {'C': 100, 'gamma': 0.01}


In [7]:
# 1. Prepare Data: SVMs expect (n_samples, n_features)
# Flattening the (16, 4) input into a 64-dimensional vector
X_train_flat = X_train_cnn.reshape(X_train_cnn.shape[0], -1)
X_test_flat = X_test_cnn.reshape(X_test_cnn.shape[0], -1)

# 2. Scaling: Crucial for SVM performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

# 3. Define and Train the SVM
# Using RBF kernel to capture non-linear relationships
svm_model = svm.SVC(
    kernel='rbf',
    C=100,          # Regularization parameter
    gamma=0.01,  # Kernel coefficient
    probability=True
)
start_xgb = time.time()
svm_model.fit(X_train_scaled, y_train)
end_xgb = time.time()

# 4. Evaluation
y_pred = svm_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print(f"Training Time: {end_xgb - start_xgb:.4f} seconds")
print(f"SVM Test Accuracy: {accuracy*100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Training Time: 29.4618 seconds
SVM Test Accuracy: 87.17%

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.95      0.93       110
           1       0.95      0.99      0.97       144
           2       0.83      0.82      0.82       115
           3       0.82      0.83      0.83       114
           4       0.84      0.82      0.83       125
           5       0.80      0.87      0.83       108
           6       0.91      0.93      0.92       110
           7       0.92      0.90      0.91       136
           8       0.84      0.80      0.82       108
           9       0.87      0.80      0.84       130

    accuracy                           0.87      1200
   macro avg       0.87      0.87      0.87      1200
weighted avg       0.87      0.87      0.87      1200



In [10]:
# Define the steps from 1000 to 10000
data_sizes = range(1000, 11000, 1000)
results = []

print(f"{'Data Size':<12} | {'Accuracy (%)':<15} | {'Time (s.)':<15}")
print("-" * 30)

for size in data_sizes:
    # Slice the training data
    X_subset = X_train_scaled[:size, :]
    y_subset = y_train[:size]

    # Re-initialize and fit (important to reset the model each time)
    start_train = time.time()
    svm_model.fit(X_subset, y_subset)
    end_train = time.time()
    time_spent = end_train - start_train
    # Evaluate
    y_pred = svm_model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred) * 100

    results.append((size, acc))
    print(f"{size:<12} | {acc:<15.2f}  | {time_spent:<15.2f}")

# Convert to array for easy plotting later
results_arr = np.array(results)

Data Size    | Accuracy (%)    | Time (s.)      
------------------------------
1000         | 70.67            | 1.06           
2000         | 76.17            | 2.44           
3000         | 79.42            | 3.33           
4000         | 81.58            | 3.45           
5000         | 82.92            | 6.22           
6000         | 84.67            | 7.38           
7000         | 85.17            | 11.27          
8000         | 85.67            | 20.68          
9000         | 86.42            | 23.85          
10000        | 87.42            | 22.90          
